In [1]:
import os
import numpy as np
import pandas as pd


In [2]:
# Where to save the figures and data files
PROJECT_ROOT_DIR = "../Results"
FIGURE_ID = "../Results/FigureFiles"
DATA_ID = "../DataFiles/"

In [3]:
if not os.path.exists(PROJECT_ROOT_DIR):
    os.mkdir(PROJECT_ROOT_DIR)

if not os.path.exists(FIGURE_ID):
    os.makedirs(FIGURE_ID)

if not os.path.exists(DATA_ID):
    os.makedirs(DATA_ID)

def image_path(fig_id):
    return os.path.join(FIGURE_ID, fig_id)

def data_path(dat_id):
    return os.path.join(DATA_ID, dat_id)

def save_fig(fig_id):
    plt.savefig(image_path(fig_id) + ".png", format='png')

In [4]:
from calculations_load import *
from configurations import *
from bayes_exp import Y_exp_data

Using idf = 0 : Grad
SystemsInfo = 
{'Pb-Pb-2760': {'proj': 'Pb', 'targ': 'Pb', 'sqrts': 2760, 'main_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat', 'main_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat', 'validation_design_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_validation_PbPb-2760.dat', 'validation_range_file': 'production_designs/500pts/design_pts_Pb_Pb_2760_production//design_ranges_validation_PbPb-2760.dat', 'labels': ['$N$[$2.76$TeV]', '$p$', '$\\sigma_k$', '$w$ [fm]', '$d_{\\mathrm{min}}^3$ [fm]', '$\\tau_R$ [fm/$c$]', '$\\alpha$', '$T_{\\eta,\\mathrm{kink}}$ [GeV]', '$a_{\\eta,\\mathrm{low}}$ [GeV${}^{-1}$]', '$a_{\\eta,\\mathrm{high}}$ [GeV${}^{-1}$]', '$(\\eta/s)_{\\mathrm{kink}}$', '$(\\zeta/s)_{\\max}$', '$T_{\\zeta,c}$ [GeV]', '$w_{\\zeta}$ [GeV]', '$\\lambda_{\\zeta}$', '$b_{\\pi}$', '$T_{\\mathrm{sw}}$ [GeV]'], '

In [5]:
system_str = 'Pb-Pb-2760'
design_file = 'production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat'
design = pd.read_csv(design_file)
design = design.drop("idx", axis=1)

#delete bad design points
drop_indices = list(delete_design_pts_set)
design = design.drop(drop_indices)

#choose features (inputs)
#feature_cols = ['norm', 'trento_p'] #specific choices
feature_cols = design.keys().values #all of them 
n_features = len(feature_cols)
X = design[feature_cols]

#perform a transformation of the design, swapping the original parameters
#for inputs more naturally/simply related to the outputs
do_transform_design = False
if do_transform_design:
    X = transform_design(X.values)

n_design = SystemsInfo["Pb-Pb-2760"]["n_design"]
npt = n_design - len(delete_design_pts_set)

Y = np.array([])
for pt in range(npt):
    for obs in active_obs_list['Pb-Pb-2760']:
        Y = np.append( Y, trimmed_model_data[system_str][pt, idf][obs]['mean'][:], axis=0)
Y = Y.reshape(X.shape[0], -1)

In [6]:
feature_cols

array(['norm', 'trento_p', 'sigma_k', 'nucleon_width', 'dmin3', 'tau_R',
       'alpha', 'eta_over_s_T_kink_in_GeV',
       'eta_over_s_low_T_slope_in_GeV', 'eta_over_s_high_T_slope_in_GeV',
       'eta_over_s_at_kink', 'zeta_over_s_max',
       'zeta_over_s_T_peak_in_GeV', 'zeta_over_s_width_in_GeV',
       'zeta_over_s_lambda_asymm', 'shear_relax_time_factor', 'Tswitch'],
      dtype=object)

In [7]:
obs_name    =  []
for obs in active_obs_list['Pb-Pb-2760']:
    for i in obs_cent_list['Pb-Pb-2760'][obs]:
        obs_name.append(f'{obs}{i}')

In [8]:
len(obs_name)

110

In [9]:
print( "X.shape : "+ str(X.shape) )
print( "Y.shape : "+ str(Y.shape) )

X.shape : (485, 17)
Y.shape : (485, 110)


In [10]:
#Write to pandas data frame
design_df=pd.DataFrame(data=X, columns=feature_cols)
simulation_out_df=pd.DataFrame(data=Y, columns=obs_name)

In [11]:
design_df.head()

,norm,trento_p,sigma_k,nucleon_width,dmin3,tau_R,alpha,eta_over_s_T_kink_in_GeV,eta_over_s_low_T_slope_in_GeV,eta_over_s_high_T_slope_in_GeV,eta_over_s_at_kink,zeta_over_s_max,zeta_over_s_T_peak_in_GeV,zeta_over_s_width_in_GeV,zeta_over_s_lambda_asymm,shear_relax_time_factor,Tswitch
0,15.81754,-0.06769,0.98153,0.92763,2.61235,1.34389,0.00988,0.21946,-0.28087,0.49299,0.10715,0.14627,0.19076,0.09729,0.15270,4.79271,0.14759
1,13.43970,0.12337,1.34278,1.10651,2.17324,0.90060,0.05149,0.23432,-0.37329,0.29940,0.08497,0.12393,0.20455,0.07722,0.29229,5.81536,0.14498
2,15.84174,0.13791,1.06155,1.15276,2.77641,1.03337,0.00518,0.23255,-0.35375,0.79445,0.13705,0.11665,0.18175,0.07408,0.07515,3.39156,0.15274
3,14.47395,0.11147,0.82684,1.01294,2.40816,1.13235,-0.08793,0.20885,-1.19838,0.43428,0.12346,0.07759,0.17377,0.06128,0.11686,4.61781,0.14550
4,16.13648,0.32229,1.38858,0.84729,1.23917,1.12739,-0.07829,0.23634,-0.22399,0.27735,0.14276,0.11019,0.20315,0.07877,-0.10924,6.42583,0.13879


In [12]:
simulation_out_df.head()

,dNch_deta[0 5],dNch_deta[ 5 10],dNch_deta[10 20],dNch_deta[20 30],dNch_deta[30 40],dNch_deta[40 50],dNch_deta[50 60],dNch_deta[60 70],dET_deta[0. 2.5],dET_deta[2.5 5. ],...,v32[10 20],v32[20 30],v32[30 40],v32[40 50],v42[0 5],v42[ 5 10],v42[10 20],v42[20 30],v42[30 40],v42[40 50]
0,1608.155095,1301.879222,953.021576,645.307009,402.420100,244.321593,138.775810,64.998956,2094.094369,1854.027153,...,0.027231,0.030437,0.032241,0.034078,0.010554,0.012550,0.014681,0.017421,0.019349,0.018653
1,1376.107143,1141.792044,896.336877,634.080481,398.155003,253.422222,146.311755,75.674266,1718.904855,1568.418976,...,0.025072,0.025309,0.027655,0.025310,0.009541,0.009735,0.011988,0.012579,0.013068,0.012530
2,1715.649143,1410.970524,1075.089936,734.002033,495.888734,315.102365,182.456817,93.457043,2165.629760,1945.260785,...,0.020419,0.021284,0.021178,0.022446,0.008585,0.009366,0.008328,0.009645,0.010275,0.010419
3,1474.812429,1198.535444,919.375636,646.697544,439.662816,280.593073,160.003266,79.338320,1927.358731,1727.315725,...,0.019061,0.020970,0.024026,0.023602,0.007337,0.008606,0.010254,0.010787,0.012219,0.011568
4,1625.855467,1369.550286,1065.824673,726.855338,504.200066,322.735247,190.869785,100.375664,2156.410538,1920.777164,...,0.025223,0.028049,0.029004,0.029121,0.010363,0.011207,0.011285,0.012834,0.012702,0.014524


In [13]:
design_df.to_csv(path_or_buf=data_path("PbPb2760_design"), index=False)

In [14]:
simulation_out_df.to_csv(path_or_buf=data_path("PbPb2760_simulation"), index=False)

In [15]:
design_ut,dsgn_min_ut,dsgn_max_ut,design_ut_lbls=load_design(system_str='Pb-Pb-2760')

Loading main points from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_points_main_PbPb-2760.dat
Loading main ranges from production_designs/500pts/design_pts_Pb_Pb_2760_production/design_ranges_main_PbPb-2760.dat
Summary of design : 


In [16]:
priors=np.array([dsgn_min_ut,dsgn_max_ut])

In [17]:
priors.shape

(2, 17)

In [18]:
prior_range=pd.DataFrame(data=priors, columns=feature_cols, index=["min", "max"])
prior_range.head()

,norm,trento_p,sigma_k,nucleon_width,dmin3,tau_R,alpha,eta_over_s_T_kink_in_GeV,eta_over_s_low_T_slope_in_GeV,eta_over_s_high_T_slope_in_GeV,eta_over_s_at_kink,zeta_over_s_max,zeta_over_s_T_peak_in_GeV,zeta_over_s_width_in_GeV,zeta_over_s_lambda_asymm,shear_relax_time_factor,Tswitch
min,10.0,-0.7,0.3,0.5,0.000,0.3,-0.3,0.13,-2.0,-1.0,0.01,0.01,0.12,0.025,-0.8,2.0,0.130
max,20.0,0.7,2.0,1.5,4.913,2.0,0.3,0.30,1.0,2.0,0.20,0.20,0.30,0.150,0.8,8.0,0.165


In [19]:
prior_range.to_csv(path_or_buf=data_path("PbPb2760_prior"))

In [20]:
Yexp=[]
Yexperr=[]

for obs in active_obs_list['Pb-Pb-2760']:
    Yexp.extend(Y_exp_data['Pb-Pb-2760'][obs]['mean'][0])
    Yexperr.extend([val**2 for val in Y_exp_data['Pb-Pb-2760'][obs]['err'][0]]) # Experimental error is given as sd. Make it variance
Yexp=np.array(Yexp)
Yexperr=np.array(Yexperr)
print(f'Shape of the experiment observables {Yexp.shape} and shape of the experimental error variance{Yexperr.shape}')

Shape of the experiment observables (110,) and shape of the experimental error variance(110,)


In [21]:
experiment=np.array([Yexp, Yexperr])
experiment.shape

(2, 110)

In [22]:
experimental_data= pd.DataFrame(data=experiment, columns=obs_name, index=["mean", "variance"])

In [23]:
experimental_data.head()

,dNch_deta[0 5],dNch_deta[ 5 10],dNch_deta[10 20],dNch_deta[20 30],dNch_deta[30 40],dNch_deta[40 50],dNch_deta[50 60],dNch_deta[60 70],dET_deta[0. 2.5],dET_deta[2.5 5. ],...,v32[10 20],v32[20 30],v32[30 40],v32[40 50],v42[0 5],v42[ 5 10],v42[10 20],v42[20 30],v42[30 40],v42[40 50]
mean,1601.0,1294.0,966.0,649.0,426.0,261.0,149.0,76.0,1803.0,1647.0,...,0.026220,0.028840,0.030310,0.030220,9.720000e-03,1.133000e-02,1.250000e-02,1.390000e-02,1.540000e-02,0.015140
variance,3600.0,2401.0,1369.0,529.0,225.0,81.0,36.0,16.0,11981.0,9882.0,...,0.000001,0.000001,0.000002,0.000002,2.362000e-07,4.234000e-07,3.229000e-07,5.345000e-07,7.093000e-07,0.000001


In [24]:
experimental_data.to_csv(path_or_buf=data_path("PbPb2760_experiment"))